## Imports

In [1]:
from gym_env.env import VVEnv
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.env_checker import check_env
import tensorboardX
import torch


pygame-ce 2.4.1 (SDL 2.28.5, Python 3.11.5)


## Configuration & Setup

### variables

In [2]:
EPOCHS = 10
N_STEPS = 1024
BATCH_SIZE = 64
N_ENVS = 8
LR = 0.0001

TOTAL_TIMESTEPS_EACH_EPOCH = 2e+5 


SCALED_TOTAL_STEPS = N_ENVS * TOTAL_TIMESTEPS_EACH_EPOCH



### Env Initialization

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f"Using device: {device}")


env = VVEnv("agent")
check_env(env)

Using device: cuda


In [4]:
vec_env = make_vec_env(VVEnv, n_envs=N_ENVS, env_kwargs={"mode": "agent"})
vec_env


## Model Steup & Training

### Model Setup

In [5]:
model = PPO("MultiInputPolicy", 
            vec_env, 
            verbose=1, 
            learning_rate=LR, 
            batch_size=BATCH_SIZE,
            n_steps=N_STEPS,
            n_epochs=EPOCHS,
            tensorboard_log="./logs/", 
            device=device)


Using cuda device


In [6]:
model.learn(total_timesteps=SCALED_TOTAL_STEPS, log_interval=1)

Logging to ./logs/PPO_7
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 350      |
|    ep_rew_mean     | -62.6    |
| time/              |          |
|    fps             | 476      |
|    iterations      | 1        |
|    time_elapsed    | 17       |
|    total_timesteps | 8192     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 398         |
|    ep_rew_mean          | -51.5       |
| time/                   |             |
|    fps                  | 336         |
|    iterations           | 2           |
|    time_elapsed         | 48          |
|    total_timesteps      | 16384       |
| train/                  |             |
|    approx_kl            | 0.010221167 |
|    clip_fraction        | 0.0419      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | 0.000329    |
|    l

In [7]:
model.save("ppo1_vv")